Our student exam numbers given for this exam 


In [4]:
import numpy as np 
import pandas as pd 

In [35]:
def importCSV():
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    all_dfs = []

    for month in months:
        for year in range(2014, 2025):
            csv_string = f"Term paper/data/SCE-{month}-{year}.csv" 
            try:
                csv_file = pd.read_csv(csv_string, sep=";")
                all_dfs.append(csv_file)
            except FileNotFoundError:
                print(f"Filen {csv_string} finnes ikke.")

    return  pd.concat(all_dfs)
df = importCSV()

In [41]:

unique_persons = df["userid"].nunique()

print(f"unique persons: {unique_persons}")
print(f"number of rows: {df.shape[0]}")

print(f"Number of unique survey waves: {df['wid'].nunique()}")

earliest_date = df['date'].min()
print(f"earliest_date: {earliest_date}")

latest_date = df['date'].max()
print(f"latest date: {latest_date}")

df.head()


unique persons: 21666
number of rows: 165924
Number of unique survey waves: 132
earliest_date: 2014-01-02
latest date: 2024-12-31


,userid,wid,date,weight,female,educ,age,hispanic,black,couple,...,num_lit_q3,num_lit_q3_correct,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct
0,70016297,201401,2014-01-07,1.0,0.0,4.0,29.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70016329,201401,2014-01-02,4.5,1.0,2.0,43.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70016366,201401,2014-01-07,0.6,0.0,4.0,30.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70016375,201401,2014-01-16,0.7,1.0,4.0,28.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70016445,201401,2014-01-19,0.5,0.0,4.0,47.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
